In [1]:
import matplotlib.pyplot as plt
import json
import os
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
data_dir = './vaccine_data/hhs_data'

In [2]:
train_years = ['2012','2013','2014']
dev_years = ['2015']
test_years = ['2016','2017']

train_download_dir = ('train',train_years)
dev_download_dir = ('dev',dev_years)
test_download_dir = ('test',test_years)
data_split_ls = [train_download_dir,dev_download_dir,test_download_dir]

In [15]:
year = 2017
request_url = "https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/{}/states/{}//{}/zipcodes/{}.json"
no_response_list = []
for index,row in county_df.iterrows():
    if index>34900:
        state_id = row["state_id"]
        county_name = row["county_name"]
        zipcodes = row["zips"]
        zipcodes = str(zipcodes).split()
        for zipcode in zipcodes:
            url = request_url.format(year,state_id,county_name,zipcode)
            response = requests.get(url)
            if response.status_code!=200:
                no_response_list.append((year,state_id,county_name,zipcode,response.status_code))
            else:
                curr_json = response.json()
                filename = "vaccine_data/{}_county_level/{}/{}/{}.json".format(year,state_id,county_name,zipcode)
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, 'w') as outfile:
                    json.dump(curr_json, outfile)
        if index%100==0:
            print("Done till index = {}".format(index))
            thefile = open('vaccine_data/{}/no_respose_list/no_response_list_till_index_{}.txt'.format(year,index), 'w')
            for item in no_response_list:
                thefile.write("{}\n".format(item))
            thefile.close()
            



Done till index = 35000
Done till index = 35100
Done till index = 35200
Done till index = 35300
Done till index = 35400
Done till index = 35500
Done till index = 35600
Done till index = 35700
Done till index = 35800
Done till index = 35900
Done till index = 36000
Done till index = 36100
Done till index = 36200
Done till index = 36300
Done till index = 36400
Done till index = 36500
Done till index = 36600
Done till index = 36700
Done till index = 36800
Done till index = 36900
Done till index = 37000
Done till index = 37100
Done till index = 37200
Done till index = 37300
Done till index = 37400


In [ ]:
# county_level
year = 2017
request_url = "https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/{}/states/{}/counties/{}.json"
no_response_list = []
for index,row in county_df.iterrows():
    if index>0:
        state_id = row["state_id"]
        county_name = row["county_name"]
        url = request_url.format(year,state_id,county_name,zipcode)
        response = requests.get(url)
        if response.status_code!=200:
            no_response_list.append((year,state_id,county_name,response.status_code))
        else:
            curr_json = response.json()
            filename = "vaccine_data/{}_county_level/{}/{}/{}.json".format(year,state_id,county_name)
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            with open(filename, 'w') as outfile:
                json.dump(curr_json, outfile)
        if index%100==0:
            print("Done till index = {}".format(index))
            thefile = open('vaccine_data/{}/no_respose_list/no_response_list_till_index_{}.txt'.format(year,index), 'w')
            for item in no_response_list:
                thefile.write("{}\n".format(item))
            thefile.close()
            



In [18]:
file_path = "vaccine_data/{}/no_respose_list/no_response_list_till_index_{}.txt".format(year,24200)

with open(file_path) as fp:  
    line = fp.readline()
    cnt = 1
    while line:
        print("Line {}: {}".format(cnt, line.strip()))
        line = fp.readline()
        cnt += 1

Line 1: (2017, 'CA', 'Modoc', '96134', 404)
Line 2: (2017, 'CA', 'San Bernardino', 'nan', 404)
Line 3: (2017, 'CA', 'Imperial', '92274', 404)
Line 4: (2017, 'CA', 'Orange', '92850', 404)
Line 5: (2017, 'CA', 'Orange', '92899', 404)
Line 6: (2017, 'CA', 'San Diego', '92155', 404)
Line 7: (2017, 'CA', 'Plumas', '96137', 404)
Line 8: (2017, 'CA', 'Tulare', '93238', 404)
Line 9: (2017, 'CA', 'Ventura', '90265', 404)
Line 10: (2017, 'CA', 'Fresno', '93760', 404)
Line 11: (2017, 'CA', 'Fresno', '93761', 404)
Line 12: (2017, 'CA', 'Fresno', '93764', 404)
Line 13: (2017, 'CA', 'Fresno', '93765', 404)
Line 14: (2017, 'CA', 'Fresno', '93844', 404)
Line 15: (2017, 'CA', 'Los Angeles', '91121', 404)
Line 16: (2017, 'CA', 'Los Angeles', '91123', 404)
Line 17: (2017, 'CA', 'Los Angeles', '91124', 404)
Line 18: (2017, 'CA', 'Los Angeles', '91126', 404)
Line 19: (2017, 'CA', 'Los Angeles', '91129', 404)
Line 20: (2017, 'CA', 'Los Angeles', '91182', 404)
Line 21: (2017, 'CA', 'Los Angeles', '91184', 40

Line 1330: (2017, 'IN', 'Vigo', '47842', 404)
Line 1331: (2017, 'IN', 'Franklin', '47060', 404)
Line 1332: (2017, 'IN', 'LaPorte', '46360', 404)
Line 1333: (2017, 'IN', 'Clay', '47438', 404)
Line 1334: (2017, 'IN', 'Franklin', '45053', 404)
Line 1335: (2017, 'IN', 'Decatur', '47261', 404)
Line 1336: (2017, 'IN', 'St. Joseph', '46574', 404)
Line 1337: (2017, 'IN', 'Porter', '46360', 404)
Line 1338: (2017, 'IN', 'Dubois', '47432', 404)
Line 1339: (2017, 'IN', 'St. Joseph', '46561', 404)
Line 1340: (2017, 'IN', 'Starke', '46574', 404)
Line 1341: (2017, 'IN', 'St. Joseph', '46637', 404)
Line 1342: (2017, 'IN', 'Orange', '47125', 404)
Line 1343: (2017, 'IN', 'DeKalb', '46706', 404)
Line 1344: (2017, 'IN', 'LaPorte', '46382', 404)
Line 1345: (2017, 'IN', 'DeKalb', '46730', 404)
Line 1346: (2017, 'IN', 'Daviess', '47553', 404)
Line 1347: (2017, 'IN', 'Rush', '46133', 404)
Line 1348: (2017, 'IN', 'Posey', '47712', 404)
Line 1349: (2017, 'IN', 'Hamilton', '46256', 404)
Line 1350: (2017, 'IN', '

In [65]:
request_url = "https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/zipcodes/19930.json"
response = requests.get(request_url)
response.status_code

404

JSONDecodeError: Expecting value: line 1 column 1 (char 0)